In [5]:
import cv2
import os
import numpy as np

In [6]:
base_dir = "./data"

sub_dirs = ['Training', 'Testing']
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

images = {'Training': {}, 'Testing': {}}
resize_dim = (224, 224)

for sub_dir in sub_dirs:
    for category in categories:
        category_path = os.path.join(base_dir, sub_dir, category)
        images[sub_dir][category] = []
        
        if os.path.exists(category_path):
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, resize_dim)
                # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                images[sub_dir][category].append(img)
        else:
            print(f"Category path does not exist: {category_path}")

In [7]:
test_glioma_images = images['Testing']['glioma']

print(f"Loaded {len(test_glioma_images)} glioma images from the Testing set.")

Loaded 300 glioma images from the Testing set.
